# STAT 5410 Final Project: Analysis of Area Codes

**Author: Mohit Bhoir**

## Introduction

This project investigates the historical allocation of area codes under the 1947 North American Numbering Plan (NANP) to uncover whether the design of telecommunications infrastructure reflected systemic social and demographic biases. Using spatial joins, historical census data from the 1950s, and statistical modeling, we map original area code assignments to U.S. counties and analyze patterns based on population size, race, and geographic distribution. Our work reveals that more populous regions were often given easier-to-dial codes, and in high-density areas, codes serving predominantly Black populations were disproportionately harder to dial. We build predictive models to forecast future area code splits using mid-century demographic and infrastructural features. Our findings demonstrate how early decisions in telecommunications policy have left a measurable imprint on long-term service distribution and social equity—offering a data-driven lens on infrastructure fairness.

In [ ]:
# Setup environment
# Note: You may need to install these packages if they're not already available
# %pip install pandas numpy matplotlib seaborn plotly geopandas scikit-learn statsmodels

# COMPONENT 1: Assigning Area Codes to Counties

In Component 1, we assembled and joined multiple datasets to map each U.S. county to its original 3-digit area code, based on 1947 assignments. This involved:

- Cleaning census, area code, and shapefile data
- Performing spatial joins to identify counties for each city
- Assigning the most common original area code per county
- Visualizing this assignment for selected states

In [ ]:
# Load all Datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from functools import reduce

# Set plotting style
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")

# Load county census data
county_census = pd.read_csv("final_project_data/county_census_info.csv")
county_census = county_census.rename(columns={
    'state_name': 'state', 
    'county_name': 'county',
    'state_fips_code': 'state_fips', 
    'county_fips_code': 'county_fips'
})

# Load merged counties data
merged_counties = pd.read_csv("final_project_data/merged_counties_since_1950.csv")
merged_counties = merged_counties.rename(columns={
    'state_name': 'state',
    'county_name': 'county'
})

# Load new counties data
new_counties = pd.read_csv("final_project_data/new_counties_since_1950.csv")
new_counties = new_counties.rename(columns={
    'new_county_name': 'county'
})

# Load area codes data
area_codes = pd.read_csv("final_project_data/cities_area_codes.csv")
area_codes = area_codes.rename(columns={
    'state_or_province': 'state'
})

# Load county shapefile data
county_shapes = gpd.read_file("final_project_data/co99_d00_shp/co99_d00.shp")
county_shapes = county_shapes.rename(columns={
    'state': 'state_fips',
    'county': 'county_fips',
    'name': 'county'
})

In [ ]:
# Process splits and overlays data
import os
import pandas as pd

# Get all sheets from the Excel file
splits_path = "final_project_data/splits_overlays.xlsx"
splits_overlays_sheets = pd.ExcelFile(splits_path).sheet_names

# Combine all sheets into one dataframe
splits_overlays_all = pd.DataFrame()
for sheet in splits_overlays_sheets:
    df = pd.read_excel(splits_path, sheet_name=sheet)
    df = df.rename(columns=lambda x: x.lower().replace(' ', '_'))
    df['original_area_code'] = int(sheet)
    splits_overlays_all = pd.concat([splits_overlays_all, df])

# Join original area codes with area codes data
area_codes_original = area_codes.copy()
area_codes_original['code'] = area_codes_original['area_code'].astype(str)
splits_overlays_all['code'] = splits_overlays_all['code'].astype(str)

area_codes_original = pd.merge(
    area_codes_original,
    splits_overlays_all[['code', 'original_area_code']],
    on='code',
    how='left'
)

In [ ]:
# Spatial join to match cities to counties
# Convert area codes to GeoDataFrame
area_codes_sf = area_codes_original[
    ~area_codes_original['latitude'].isna() & 
    ~area_codes_original['longitude'].isna()
].copy()

area_codes_sf = gpd.GeoDataFrame(
    area_codes_sf, 
    geometry=gpd.points_from_xy(area_codes_sf.longitude, area_codes_sf.latitude),
    crs="EPSG:4326"
)

# Set CRS for county shapes
county_shapes = county_shapes.set_crs("EPSG:4269")

# Transform county shapes to match area codes CRS
county_shapes_transformed = county_shapes.to_crs(area_codes_sf.crs)

# Perform spatial join
cities_with_counties = gpd.sjoin(area_codes_sf, county_shapes_transformed, how="inner")

# Find most common original area code per county
county_area_codes = cities_with_counties.drop(columns='geometry').groupby(['state_fips', 'county_fips'])
county_area_codes = county_area_codes['original_area_code'].agg(
    lambda x: x.value_counts().index[0] if not x.isna().all() else None
).reset_index()
county_area_codes.columns = ['state_fips', 'county_fips', 'original_area_code']

# Join to shapefile
county_shapes_with_area_code = county_shapes.merge(
    county_area_codes, 
    on=['state_fips', 'county_fips'],
    how='left'
)

county_map_data = county_shapes_with_area_code.merge(
    county_census[['state_fips', 'county_fips', 'state']],
    on=['state_fips', 'county_fips'],
    how='left'
)

In [ ]:
# Function to plot state maps
def plot_state_map(state_name):
    state_data = county_map_data[county_map_data['state'] == state_name]
    
    fig, ax = plt.subplots(figsize=(12, 8))
    state_data.plot(
        column='original_area_code', 
        categorical=True,
        legend=True,
        ax=ax,
        edgecolor='white',
        linewidth=0.1
    )
    
    ax.set_title(f"Original Area Codes in {state_name}", fontsize=15)
    ax.set_axis_off()
    plt.tight_layout()
    return fig

# Plot maps for selected states
states_to_plot = ["CALIFORNIA", "NEW YORK", "TEXAS", "ILLINOIS", "KANSAS", "CONNECTICUT"]
for state in states_to_plot:
    plot_state_map(state)

The maps show distinct patterns of original area code assignment across states. Urbanized states such as New York and California exhibit dense clusters of area codes, especially around major cities like New York City and Los Angeles. In contrast, rural states like Kansas have larger, more uniformly distributed area code zones. These visualizations not only confirm the success of my spatial join and aggregation process, but also closely match the historical area code maps provided in the reference links within the project prompt, further validating the accuracy of the implementation.

---

# COMPONENT 2: Summarizing Regional Characteristics by Area Code

In this component, we constructed a summary dataset for each original area code based on demographic, telephone, and geographic information from the 1940s–1950s. This data provides the foundation for later modeling and fairness analysis.

Specifically, we:

- Aggregated census and area variables from the county to the area code level
- Calculated region-wide totals and ratios (e.g., birth rate, Black population proportion)
- Joined with descendant area code counts for future modeling

In [ ]:
# Calculate county area
county_area_km = county_shapes.drop(columns='geometry').groupby(['state_fips', 'county_fips'])
county_area_km = county_area_km['area'].sum().reset_index()

# Join area to county census
county_census = county_census.merge(
    county_area_km, 
    on=['state_fips', 'county_fips'],
    how='left'
)

In [ ]:
# Build area code summary statistics
area_code_summary = county_census.merge(
    county_area_codes,
    on=['state_fips', 'county_fips'],
    how='inner'
)

area_code_summary = area_code_summary[~area_code_summary['original_area_code'].isna()]

# Group by area code and calculate summary statistics
area_code_summary = area_code_summary.groupby('original_area_code').agg({
    'state': 'first',
    'area': 'sum',
    'population_1950': 'sum',
    'residence_telephones_1945': 'sum',
    'population_under5_1950': 'sum',
    'population_over65_1950': 'sum',
    'births': 'sum',
    'black_pop': 'sum'
}).reset_index()

# Calculate Black population proportion
area_code_summary['black_prop'] = area_code_summary['black_pop'] / area_code_summary['population_1950']

# Add middle code digit (for later analysis)
area_code_summary['original_code'] = area_code_summary['original_area_code']
area_code_summary['middle_code'] = area_code_summary['original_area_code'].astype(str).str[1]

# Join descendant counts
descendant_counts = splits_overlays_all.groupby('original_area_code').size().reset_index()
descendant_counts.columns = ['original_area_code', 'n_descendants']

area_code_summary = area_code_summary.merge(
    descendant_counts,
    left_on='original_code',
    right_on='original_area_code',
    how='left'
)

area_code_summary['n_descendants'] = area_code_summary['n_descendants'].fillna(0)

In [ ]:
# Check the summary
area_code_summary[area_code_summary['middle_code'] == '1'].sort_values(
    by='population_1950', ascending=False
)[['original_code', 'population_1950']].head(3)

In [ ]:
# Plot telephones vs population
plt.figure(figsize=(10, 6))
plt.scatter(
    area_code_summary['population_1950'], 
    area_code_summary['residence_telephones_1945'],
    color='steelblue', 
    alpha=0.6,
    s=60
)
plt.title('Residential Telephones vs Population (1950)', fontsize=14)
plt.xlabel('Population (1950)')
plt.ylabel('Residential Telephones (1945)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Plot top 10 area codes by Black population proportion
top10_black = area_code_summary.sort_values('black_prop', ascending=False).head(10)

plt.figure(figsize=(12, 6))
plt.barh(
    top10_black['original_code'].astype(str),
    top10_black['black_prop'],
    color='darkred'
)
plt.title('Top 10 Area Codes by Black Population Proportion (1950)', fontsize=14)
plt.xlabel('Proportion Black')
plt.ylabel('Original Area Code')
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

In [ ]:
# Plot distribution of births by middle digit
plt.figure(figsize=(12, 6))
sns.boxplot(x='middle_code', y='births', data=area_code_summary, color='lightblue')
plt.title('Distribution of Births by Middle Digit of Area Code', fontsize=14)
plt.xlabel('Middle Digit')
plt.ylabel('Total Births (1950)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---

# COMPONENT 3: Dialing Effort and Population – Stratified Analysis

This component evaluates whether more populous areas were intentionally assigned area codes that required fewer rotary dial pulls, especially within states that had multiple area codes to choose from.

We split states into:

- Multi-Code States (states assigned more than one area code in 1947)
- Single-Code States (states with only one assigned code)

In [ ]:
# Compute dial pulls for each area code
dial_pulls_table = pd.DataFrame({
    'digit': range(10),
    'pulls': [10, 1, 2, 3, 4, 5, 6, 7, 8, 9]
})

# Extract digits from area codes
area_code_summary['digit1'] = area_code_summary['original_code'].astype(str).str[0].astype(int)
area_code_summary['digit2'] = area_code_summary['original_code'].astype(str).str[1].astype(int)
area_code_summary['digit3'] = area_code_summary['original_code'].astype(str).str[2].astype(int)

# Join with dial pulls table
area_code_summary = area_code_summary.merge(
    dial_pulls_table.rename(columns={'pulls': 'pulls1'}),
    left_on='digit1',
    right_on='digit',
    how='left'
).drop(columns='digit')

area_code_summary = area_code_summary.merge(
    dial_pulls_table.rename(columns={'pulls': 'pulls2'}),
    left_on='digit2',
    right_on='digit',
    how='left'
).drop(columns='digit')

area_code_summary = area_code_summary.merge(
    dial_pulls_table.rename(columns={'pulls': 'pulls3'}),
    left_on='digit3',
    right_on='digit',
    how='left'
).drop(columns='digit')

# Calculate total dial pulls
area_code_summary['dial_pulls'] = area_code_summary['pulls1'] + area_code_summary['pulls2'] + area_code_summary['pulls3']

In [ ]:
# Verify aggregation
print(f"Total Population: {area_code_summary['population_1950'].sum():,.0f}")
print(f"Total Area: {area_code_summary['area'].sum():,.0f} km²")

In [ ]:
# Identify states with multiple original area codes
area_code_counts = county_area_codes.merge(
    county_census[['state_fips', 'state']].drop_duplicates(),
    on='state_fips',
    how='left'
)

area_code_counts = area_code_counts[['state', 'original_area_code']].drop_duplicates()

multi_code_states = area_code_counts.groupby('state').size().reset_index()
multi_code_states.columns = ['state', 'n']
multi_code_states['type'] = np.where(multi_code_states['n'] > 1, 'Multi-Code', 'Single-Code')

# Join to area code summary
area_code_summary = area_code_summary.merge(
    multi_code_states,
    left_on='state',
    right_on='state',
    how='left'
)

In [ ]:
# Correlation in multi-code vs single-code states
correlation_results = area_code_summary.groupby('type').apply(
    lambda x: pd.Series({
        'cor': np.corrcoef(x['population_1950'], x['dial_pulls'])[0, 1]
    })
).reset_index()

correlation_results

In [ ]:
# Plot relationship between dial pulls and 1950 population
plt.figure(figsize=(12, 7))
for type_name, group in area_code_summary.groupby('type'):
    plt.scatter(
        group['dial_pulls'],
        group['population_1950'] / 1e6,
        label=type_name,
        alpha=0.7,
        s=70
    )
    
    # Add regression line
    x = group['dial_pulls']
    y = group['population_1950'] / 1e6
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    plt.plot(x, p(x), '--')

plt.title('Dial Pulls vs Population, by Area Code Type', fontsize=14)
plt.xlabel('Dial Pulls Required')
plt.ylabel('Population (Millions)')
plt.legend(title='State Type')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Residual analysis to find unfairly slow or fast codes
residuals_df = pd.DataFrame()

for type_name, group in area_code_summary.groupby('type'):
    # Fit linear model
    X = group['population_1950'].values.reshape(-1, 1)
    y = group['dial_pulls'].values
    
    from sklearn.linear_model import LinearRegression
    model = LinearRegression().fit(X, y)
    
    # Get predictions and residuals
    predicted = model.predict(X)
    residual = y - predicted
    
    # Create dataframe with results
    group_df = group.copy()
    group_df['predicted'] = predicted
    group_df['residual'] = residual
    residuals_df = pd.concat([residuals_df, group_df])

# Find unfairly slow codes (high positive residuals)
unfair_slow = residuals_df.groupby('type').apply(
    lambda x: x.nlargest(3, 'residual')
).reset_index(drop=True)
unfair_slow['label'] = 'Unfairly Slow'

# Find unfairly fast codes (high negative residuals)
unfair_fast = residuals_df.groupby('type').apply(
    lambda x: x.nsmallest(3, 'residual')
).reset_index(drop=True)
unfair_fast['label'] = 'Unfairly Fast'

# Combine unfair codes
unfair_codes = pd.concat([unfair_slow, unfair_fast])

In [ ]:
# Plot unfair codes
plt.figure(figsize=(14, 8))

# Create a categorical color map
colors = {'Unfairly Slow': 'firebrick', 'Unfairly Fast': 'steelblue'}

# Plot for Multi-Code
plt.subplot(1, 2, 1)
multi_codes = unfair_codes[unfair_codes['type'] == 'Multi-Code']
bars = plt.barh(
    multi_codes['original_code'].astype(str),
    multi_codes['residual'],
    color=multi_codes['label'].map(colors)
)

# Add population labels
for i, v in enumerate(multi_codes['residual']):
    plt.text(
        v/2 if v > 0 else v/2, 
        i, 
        f"{multi_codes['population_1950'].iloc[i]/1e6:.1f}M",
        ha='center',
        va='center'
    )
    
plt.title('Multi-Code States')
plt.xlabel('Residual (Dial Pulls - Expected)')
plt.ylabel('Original Area Code')

# Plot for Single-Code
plt.subplot(1, 2, 2)
single_codes = unfair_codes[unfair_codes['type'] == 'Single-Code']
bars = plt.barh(
    single_codes['original_code'].astype(str),
    single_codes['residual'],
    color=single_codes['label'].map(colors)
)

# Add population labels
for i, v in enumerate(single_codes['residual']):
    plt.text(
        v/2 if v > 0 else v/2, 
        i, 
        f"{single_codes['population_1950'].iloc[i]/1e6:.1f}M",
        ha='center',
        va='center'
    )

plt.title('Single-Code States')
plt.xlabel('Residual (Dial Pulls - Expected)')

plt.suptitle('Unfairly Slow vs Fast Area Codes (Based on Dial Pull Residuals)', fontsize=14)

# Add a common legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], color=colors['Unfairly Slow'], lw=4, label='Unfairly Slow'),
    Line2D([0], [0], color=colors['Unfairly Fast'], lw=4, label='Unfairly Fast')
]
plt.figlegend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, 0.05), ncol=2)

plt.tight_layout(rect=[0, 0.1, 1, 0.95])
plt.show()

---

# COMPONENT 4: Racial Disparities in Dialing Effort

In this section, we investigate whether AT&T's 1947 area code assignments were discriminatory toward regions with higher proportions of Black residents by assigning them codes that were slower to dial on rotary phones.

To assess this, we stratify the data into High Pop and Low Pop groups (based on the median 1950 population) and fit separate linear regression models of dialing effort on Black population proportion.

In [ ]:
# Create population groups
area_code_summary['pop_group'] = np.where(
    area_code_summary['population_1950'] >= area_code_summary['population_1950'].median(),
    'High Pop',
    'Low Pop'
)

# Fit regression models within each group
import statsmodels.api as sm

stratified_models = []
for group_name, group_data in area_code_summary.groupby('pop_group'):
    X = sm.add_constant(group_data['black_prop'])
    y = group_data['dial_pulls']
    model = sm.OLS(y, X).fit()
    
    # Store results
    results = pd.DataFrame({
        'group': [group_name],
        'term': ['black_prop'],
        'estimate': [model.params['black_prop']],
        'std.error': [model.bse['black_prop']],
        'statistic': [model.tvalues['black_prop']],
        'p.value': [model.pvalues['black_prop']]
    })
    
    stratified_models.append(results)

stratified_models = pd.concat(stratified_models)
stratified_models

In [ ]:
# Plot regression results
plt.figure(figsize=(12, 7))

for group_name, group_data in area_code_summary.groupby('pop_group'):
    plt.scatter(
        group_data['black_prop'],
        group_data['dial_pulls'],
        label=group_name,
        alpha=0.6,
        s=70
    )
    
    # Add regression line
    x = group_data['black_prop']
    y = group_data['dial_pulls']
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    x_range = np.linspace(x.min(), x.max(), 100)
    plt.plot(x_range, p(x_range), '--')

plt.title('Stratified Regression by Population Group', fontsize=14)
plt.xlabel('Proportion Black (1950)')
plt.ylabel('Dial Pulls')
plt.legend(title='Population Group')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---

# COMPONENT 5: Predicting Area Code Growth

### Note on Evaluation Metric: Poisson Log Loss

In this component, we aimed to predict how many descendant area codes each original code would eventually generate—a measure of telecommunication demand and population pressure.

Although our primary evaluation metrics were RMSE and R², it's important to recognize that `n_descendants` is a count response variable. For count data, the **Poisson log loss** (also known as **Poisson deviance**) is a more appropriate loss function. It better reflects the likelihood-based fit for count outcomes.

The number of descendant area codes for each original code reflects long-term telecommunication demand. Using 1950-era features (e.g. population, phone usage, demographics), we seek to predict this count using multiple statistical and machine learning models.

In [ ]:
# Define Poisson log loss function
def poisson_log_loss(y_true, y_pred):
    return np.sum(y_pred - y_true * np.log(y_pred + 1e-10))

In [ ]:
# Prepare dataset for modeling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Prepare the dataset
area_data = area_code_summary.drop(columns=['original_code']).copy()
area_data['middle_code'] = area_data['middle_code'].astype('category')
area_data = area_data.dropna()

# Split data
np.random.seed(123)
train_idx, test_idx = train_test_split(area_data.index, test_size=0.2)
area_train = area_data.loc[train_idx]
area_test = area_data.loc[test_idx]

In [ ]:
# Prepare features and target
feature_cols = [col for col in area_train.columns if col not in ['n_descendants', 'state']]
X_train = area_train[feature_cols]
y_train = area_train['n_descendants']
X_test = area_test[feature_cols]
y_test = area_test['n_descendants']

# Handle categorical variables
X_train = pd.get_dummies(X_train, columns=['middle_code', 'type', 'pop_group'], drop_first=True)
X_test = pd.get_dummies(X_test, columns=['middle_code', 'type', 'pop_group'], drop_first=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Standardize numeric features
numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

In [ ]:
# Linear regression baseline
from sklearn.linear_model import LinearRegression

lm_model = LinearRegression()
lm_model.fit(X_train, y_train)
lm_preds = lm_model.predict(X_test)

lm_rmse = np.sqrt(mean_squared_error(y_test, lm_preds))
lm_r2 = r2_score(y_test, lm_preds)

print(f"Linear Regression - RMSE: {lm_rmse:.3f}, R²: {lm_r2:.3f}")

In [ ]:
# Define alternative models
from sklearn.linear_model import Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import PoissonRegressor

models = {
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=1.0),
    'KNN': KNeighborsRegressor(n_neighbors=5),
    'Tree': DecisionTreeRegressor(),
    'Boosted': XGBRegressor(),
    'Poisson': PoissonRegressor()
}

# Fit models and evaluate
model_results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    r2 = r2_score(y_test, preds)
    
    model_results.append({
        'model': name,
        'rmse': rmse,
        'rsq': r2
    })

# Add linear model results
model_results.append({
    'model': 'Linear',
    'rmse': lm_rmse,
    'rsq': lm_r2
})

# Compare all models
results_df = pd.DataFrame(model_results).sort_values('rmse')
results_df

In [ ]:
# Stepwise regression
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Convert dataframe to format for statsmodels
train_data_sm = area_train.drop(columns=['state'])
test_data_sm = area_test.drop(columns=['state'])

# Convert categorical variables
train_data_sm = pd.get_dummies(train_data_sm, columns=['middle_code', 'type', 'pop_group'], drop_first=True)
test_data_sm = pd.get_dummies(test_data_sm, columns=['middle_code', 'type', 'pop_group'], drop_first=True)
test_data_sm = test_data_sm.reindex(columns=train_data_sm.columns, fill_value=0)

# Full model
X_sm = sm.add_constant(train_data_sm.drop(columns=['n_descendants']))
y_sm = train_data_sm['n_descendants']
full_model = sm.OLS(y_sm, X_sm).fit()

# Forward stepwise selection (simplified implementation)
from sklearn.feature_selection import SequentialFeatureSelector

step_selector = SequentialFeatureSelector(
    LinearRegression(),
    n_features_to_select=6,  # Simplified - normally would use a criterion
    direction='forward'
)

step_selector.fit(X_train, y_train)
selected_features = X_train.columns[step_selector.get_support()].tolist()

# Fit model with selected features
X_train_step = X_train[selected_features]
X_test_step = X_test[selected_features]

step_model = LinearRegression().fit(X_train_step, y_train)
step_preds = step_model.predict(X_test_step)

step_rmse = np.sqrt(mean_squared_error(y_test, step_preds))
step_r2 = r2_score(y_test, step_preds)

print(f"Stepwise Model - RMSE: {step_rmse:.3f}, R²: {step_r2:.3f}")
print(f"Selected features: {selected_features}")

In [ ]:
# Plot predicted vs actual
plt.figure(figsize=(10, 6))

plt.scatter(y_test, lm_preds, label='Full Linear Model', alpha=0.7, s=70)
plt.scatter(y_test, step_preds, label='Stepwise Model', alpha=0.7, s=70)
plt.plot([0, y_test.max()], [0, y_test.max()], 'k--')

plt.title('Predicted vs Actual for Full vs Stepwise Models', fontsize=14)
plt.xlabel('Actual Descendant Count')
plt.ylabel('Predicted')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Variable importance from boosted model
boost_model = XGBRegressor()
boost_model.fit(X_train, y_train)

# Get feature importance
importance = boost_model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importance
}).sort_values('Importance', ascending=False).head(10)

plt.figure(figsize=(12, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.title('Feature Importance from Boosted Tree Model', fontsize=14)
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

In [ ]:
# Convert to binary classification (high vs low growth)
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score

# Define high growth as 2+ descendants
y_test_binary = (y_test >= 2).astype(int)
step_preds_binary = (step_preds >= 2).astype(int)

# Calculate metrics
conf_mat = confusion_matrix(y_test_binary, step_preds_binary)
accuracy = accuracy_score(y_test_binary, step_preds_binary)
sensitivity = recall_score(y_test_binary, step_preds_binary)
specificity = recall_score(1 - y_test_binary, 1 - step_preds_binary)

print("Confusion Matrix:")
print(conf_mat)
print(f"\nAccuracy: {accuracy:.3f}")
print(f"Sensitivity: {sensitivity:.3f}")
print(f"Specificity: {specificity:.3f}")

---

# Conclusion

## Key Findings

- **Population matters**: More populated areas received easier codes, especially in states with multiple code options.
- **Racial disparities exist**: Black-majority areas in urban states were assigned more difficult-to-dial codes.
- **Births predict growth**: Early birth rates are strongly predictive of long-term telecom demand.
- **Simple models work**: Despite testing complex algorithms, a stepwise linear model outperformed them all.

## Summary

This project used spatial joins, demographic aggregation, correlation analysis, and predictive modeling to reverse-engineer the logic behind the 1947 area code map. Along the way, we uncovered compelling evidence of bias both in favor of high-population areas and against predominantly Black communities. Perhaps more importantly, we showed that early indicators like births and phone ownership were not only used to justify code assignments but could successfully forecast how those codes would evolve.

While the dataset reflects a specific moment in U.S. infrastructure history, the lessons extend far beyond telecom. It reminds us that design decisions—even numeric ones—can have long-lasting social consequences. And with the right tools, we can make those patterns visible.